In [ ]:
import torch
from torchvision.datasets import CocoDetection
from torch.utils.data.dataloader import default_collate
import os

import pathlib
import numpy as np
import PIL
import json


from torchvision.transforms import ToTensor
import albumentations as A
import torchvision.transforms as T
import cv2

import random
# from PIL import Image

# import torch
import torchvision.transforms as transforms
from torchvision.ops.boxes import box_iou

from utils import Encoder


In [ ]:
def find_files(dir_path, patterns=[None], exclusive_patterns=[None]):
    """
    Returns a generator yielding files matching the given patterns

    dir_path: Directory to search for files under. Defaults to current dir.
    patterns: Patterns of files to search for. Defaults to ["*"]. Example: ["*.json", "*.xml"]
    exclusive patterns: patterns of files not to serach for. Defaults to [None]

    """

    path = dir_path
    inclusive_path_patterns = patterns
    exclusive_path_patterns = exclusive_patterns
    all_files = pathlib.Path(dir_path)

    filtered_set = set()

    for pattern in inclusive_path_patterns:
        filtered_set = filtered_set.union(set(all_files.rglob(pattern)))

    for exclusive in exclusive_path_patterns:
        if exclusive == None:

            filtered_set = (file for file in filtered_set)

        else:

            filtered_set = filtered_set - set(all_files.rglob(exclusive))

    # filtered_gen = (file for file in np.sort(np.array(list(filtered_set))))

    return sorted(list(filtered_set))

In [ ]:
def read_rois_from_json(json_file):

    with open(json_file, 'r') as f:
        try:
            roi_file = json.load(f)


        except Exception:
            print("Error reading json file")
    rois = roi_file['rois']

    rois= np.array(rois).squeeze()[:,:]


    return rois


In [ ]:

def read_rois_from_txt(txt_file):

    bboxes = []
    with open(txt_file, 'r') as f:
        # numpy_test = np.array(f.readlines())
        for line in f.readlines():
            bboxes.append(line.strip().split(' '))

    rois = np.array(bboxes, dtype = float)

    return rois


In [ ]:
imgs = find_files('/home/sk/repo/datasets/Denovix/images/train/', ['*.png'])
txts = find_files('/home/sk/repo/datasets/Denovix/labels/train/', ['*.txt'])

In [ ]:
import PIL.Image as Image
image = Image.open(imgs[0])

In [ ]:
target = read_rois_from_txt(txts[0])


In [ ]:
rois = target[:, 1:].astype(np.float64)
class_labels = target[:, 0].astype(np.float64)


In [ ]:

album_transform = A.Compose([
# A.RandomCrop(width=450, height=450),
A.HorizontalFlip(p=0.5),
A.RandomRotate90(p=0.25),
# A.ColorJitter(p=0.25, brightness= .2, contrast=0, saturation=0, hue= .5),
# A.Resize(300, 300)
], bbox_params=A.BboxParams(format='coco', min_area=0, min_visibility=0.7, label_fields=['class_labels']))

# transformed = album_transform(image=np.asarray(image), bboxes=rois, class_labels=class_labels)


In [ ]:
ttt = album_transform(image=np.asarray(image), bboxes=rois, class_labels=class_labels)


In [ ]:
rois

In [ ]:
ttt['bboxes']

In [ ]:
rois = target[:, 1:].astype(np.float64)
class_labels = target[:, 1].astype(np.float64)

devider = np.zeros_like(rois) 

rois_copy = rois.copy()

devider[:, [0, 2]] = image.width
devider[:, [1, 3]] = image.height

rois[:, 2] = rois_copy[:, 0] + rois_copy[:, 2] # get bbox x + width
rois[:, 3] = rois_copy[:, 1] + rois_copy[:, 3]  # get bbox y + height
normalized_rois = rois / devider  # normalize the bbox 


In [ ]:
torch.tensor(class_labels, dtype = torch.long)

In [ ]:
= torch.tensor(class_labels, dtype = torch.long)

In [ ]:
class_labels

In [ ]:
torch.tensor(normalized_rois, dtype = torch.float)

In [ ]:
        

        
#         if self.img_aug:
            
#             rois = rois.tolist()
#             class_labels = class_labels.tolist()


#             transformed = self.img_aug(image=np.asarray(image), bboxes=rois, class_labels=class_labels)

#             transformed_image = transformed['image']
#             transformed_bboxes = transformed['bboxes']
#             transformed_class_labels = transformed['class_labels']        

#             ######### normalize rois##############

#             transformed_bboxes_np = np.array(transformed_bboxes)

#             devider = np.zeros_like(transformed_bboxes_np) 

#             devider[:, [0, 2]] = float(image.width)
#             devider[:, [1, 3]] = float(image.height)

#             transformed_bboxes_np[:, 2] = transformed_bboxes_np[:, 0] + transformed_bboxes_np[:, 2] # get bbox x + width
#             transformed_bboxes_np[:, 3] = transformed_bboxes_np[:, 1] + transformed_bboxes_np[:, 3]  # get bbox y + height
#             normalized_transformed_bboxes_np = transformed_bboxes_np / devider  # normalize the bbox 

            
#             bboxes, labels = self.encoder.encode(torch.tensor(normalized_transformed_bboxes_np, dtype = torch.float), torch.tensor(transformed_class_labels).long())        

#             tensor_img = transform(transformed_image).repeat(3, 1, 1) # make 3 channel images from gray image in tensor
        
#         else:
img_resizer= transforms.Resize(300)


In [ ]:
class SSD_Dataset(object):
    def __init__(self, img_path_list, target_path_list, dboxes, img_aug=None):
        # nothing special here, just internalizing the constructor parameters

        self.img_aug = img_aug
        self.img_path_list = img_path_list
        self.target_path_list = target_path_list
        self.dboxes = dboxes
        self.encoder = Encoder(self.dboxes)
                
    def __len__(self):
        return len(self.img_path_list)
    
    def _read_rois_from_json(self, json_file):
        
        with open(json_file, 'r') as f:
            try:
                roi_file = json.load(f)
                

            except Exception:
                print("Error reading json file")
        rois = roi_file['rois']

        rois= np.array(rois).squeeze()[:,:]


        return rois

    def __getitem__(self, index):
        
        transform = ToTensor()
        
        image = PIL.Image.open(self.img_path_list[index])
        
        
                
        target = self._read_rois_from_json(self.target_path_list[index])
        
        rois = target[:, 1:].tolist()
        class_labels = target[:, 1].tolist()
        
        
        
        album_transform = A.Compose([
    A.RandomCrop(width=450, height=450),
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.25),
    A.ColorJitter(p=0.25, brightness= .2, contrast=0, saturation=0, hue= .5),
    A.Resize(300, 300)
], bbox_params=A.BboxParams(format='coco', min_area=30, min_visibility=0.1, label_fields=['class_labels']))
        
        transformed = album_transform(image=np.asarray(image), bboxes=rois, class_labels=class_labels)
        
        transformed_image = transformed['image']
        transformed_bboxes = transformed['bboxes']
        transformed_class_labels = transformed['class_labels']        
        
        ######### normalize rois##############
        
        transformed_bboxes_np = np.array(transformed_bboxes)
        
        devider = np.zeros_like(transformed_bboxes_np) 
            
        devider[:, [0, 2]] = float(image.width)
        devider[:, [1, 3]] = float(image.height)

        transformed_bboxes_np[:, 2] = transformed_bboxes_np[:, 0] + transformed_bboxes_np[:, 2] # get bbox x + width
        transformed_bboxes_np[:, 3] = transformed_bboxes_np[:, 1] + transformed_bboxes_np[:, 3]  # get bbox y + height
        normalized_transformed_bboxes_np = transformed_bboxes_np / devider  # normalize the bbox 
        
        bboxes, labels = self.encoder.encode(torch.tensor(normalized_transformed_bboxes_np, dtype = torch.float), torch.tensor(transformed_class_labels).long())        
        
        tensor_img = transform(transformed_image).repeat(3, 1, 1)
        
        return (tensor_img, bboxes, labels)



In [ ]:
target_path_list = find_files('/home/sk/Rewire_Image/Rewire_original_models/c-fos/', ['*.json'])
img_path_list = find_files('/home/sk/Rewire_Image/Rewire_original_models/c-fos/', ['*.jpg'])

In [ ]:
import utils
dbbox = utils.generate_dboxes(model="ssd")

In [ ]:
dataset = SSD_Dataset(img_path_list=img_path_list, target_path_list=target_path_list, dboxes=dbbox)

In [ ]:
import torch
from torchvision.datasets import CocoDetection
from torch.utils.data.dataloader import default_collate
import os
import pathlib
import numpy as np
import PIL
import json


from torchvision.transforms import ToTensor
import albumentations as A

from tqdm.autonotebook  import tqdm


In [ ]:


def collate_fn(batch):
    items = list(zip(*batch))
    items[0] = default_collate([i for i in items[0] if torch.is_tensor(i)])
    # items[1] = list([i for i in items[1] if i])
    # items[2] = list([i for i in items[2] if i])
    items[1] = default_collate([i for i in items[1] if torch.is_tensor(i)])
    items[2] = default_collate([i for i in items[2] if torch.is_tensor(i)])
    return items


class CocoDataset(CocoDetection):
    def __init__(self, root, mode, transform=None):
        annFile = os.path.join(root, "annotations", "instances_{}.json".format(mode))
        root = os.path.join(root, "{}".format(mode))
        super(CocoDataset, self).__init__(root, annFile)
        self._load_categories()
        self.transform = transform

    def _load_categories(self):

        categories = self.coco.loadCats(self.coco.getCatIds())
        categories.sort(key=lambda x: x["id"])

        self.label_map = {}
        self.label_info = {}
        counter = 1
        self.label_info[0] = "background"
        for c in categories:
            self.label_map[c["id"]] = counter
            self.label_info[counter] = c["name"]
            counter += 1

    def __getitem__(self, item):
        image, target = super(CocoDataset, self).__getitem__(item)
        width, height = image.size
        boxes = []
        labels = []
        if len(target) == 0:
            return None, None, None, None, None
        for annotation in target:
            bbox = annotation.get("bbox")
            # boxes.append([bbox[0] / width, bbox[1] / height, (bbox[0] + bbox[2]) / width, (bbox[1] + bbox[3]) / height])
            boxes.append([bbox[0], bbox[1], bbox[2], bbox[3]])
            labels.append(self.label_map[annotation.get("category_id")])
        boxes = torch.tensor(np.array(boxes))
        labels = torch.tensor(labels)
        if self.transform is not None:
            album_transform = A.Compose([
                # A.RandomCrop(width=450, height=450),
                # A.CenterCrop(p=1, height=height, width=width)],

                A.LongestMaxSize(300),
                A.HorizontalFlip(p=0.5),
                A.RandomRotate90(p=0.25),
                A.ColorJitter(p=0.25, brightness= .2, contrast=0, saturation=0, hue= .5),
                A.PadIfNeeded(min_height=300, min_width=300, p=1)],
                bbox_params=A.BboxParams(format='coco', min_area=1, min_visibility=0.05, label_fields=['class_labels']))
            
            transformed = album_transform(image=np.asarray(image), bboxes=boxes, class_labels=labels)
                  
            image = transformed['image']
            boxes = transformed['bboxes']
            for i, box in enumerate(boxes):
                boxes[i] = [box[0] / image.shape[0], box[1] / image.shape[1], (box[0] + box[2]) / image.shape[0], (box[1] + box[3]) / image.shape[1]]

            labels = transformed['class_labels']
        return (image, boxes, labels)

In [ ]:
train_data = CocoDataset('/home/sk/repo/FishCam/splitted_anno_0/', 'train', transform=True)

In [ ]:
import matplotlib.pylab as plt

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_dataloader =DataLoader(train_data)

In [ ]:
train_bar = tqdm(train_dataloader, desc=f"batch Loss: ", leave=False)
for i, (img, gloc, glabel) in enumerate(train_bar):
    print(i)
    # break

In [ ]:
train_data = CocoDataset('/home/sk/repo/FishCam/splitted_anno_0/', 'train', transform=True)
train_dataloader= DataLoader(train_data)

In [ ]:
train_data[3][0].shape

In [ ]:
plt.imshow(train_data[3][0])

In [ ]:
train_data[3][1]

In [ ]:
Datal

In [ ]:
val_data[0][2].get('category_id')

In [ ]:
val_data[0][4]

In [ ]:
import json
  
# Opening JSON file
f = open('/home/sk/SSD-pytorch/coco/annotations/instances_train2017.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Iterating through the json
# list
for i in data['emp_details']:
    print(i)
  
# Closing file
f.close()

In [ ]:
self.train_set = src.dataset.SSD_Dataset(self.img_train_list, self.target_train_list, self.dboxes, img_aug = self.img_aug)
self.train_loader = DataLoader(self.train_set, batch_size = self.batch_size, shuffle = True, num_workers = self.num_workers)

In [ ]:
class SSDDataset():
    def __init__(self, json_path, transform=None):
        annFile = os.path.join(root, "annotations", "instances_{}{}.json".format(mode, year))
        root = os.path.join(root, "{}{}".format(mode, year))
        super(CocoDataset, self).__init__(root, annFile)
        self._load_categories()
        self.transform = transform

    def _load_categories(self):

        categories = self.coco.loadCats(self.coco.getCatIds())
        categories.sort(key=lambda x: x["id"])

        self.label_map = {}
        self.label_info = {}
        counter = 1
        self.label_info[0] = "background"
        for c in categories:
            self.label_map[c["id"]] = counter
            self.label_info[counter] = c["name"]
            counter += 1

    def __getitem__(self, item):
        image, target = super(CocoDataset, self).__getitem__(item)
        width, height = image.size
        boxes = []
        labels = []
        if len(target) == 0:
            return None, None, None, None, None
        for annotation in target:
            bbox = annotation.get("bbox")
            boxes.append([bbox[0] / width, bbox[1] / height, (bbox[0] + bbox[2]) / width, (bbox[1] + bbox[3]) / height])
            labels.append(self.label_map[annotation.get("category_id")])
        boxes = torch.tensor(boxes)
        labels = torch.tensor(labels)
        if self.transform is not None:
            image, (height, width), boxes, labels = self.transform(image, (height, width), boxes, labels)
        # return image, target[0]["image_id"], (height, width), boxes, labels
        return target

In [ ]:
import pathlib

In [ ]:
def find_files(dir_path, patterns=[None], exclusive_patterns=[None]):
    """
    Returns a generator yielding files matching the given patterns

    dir_path: Directory to search for files under. Defaults to current dir.
    patterns: Patterns of files to search for. Defaults to ["*"]. Example: ["*.json", "*.xml"]
    exclusive patterns: patterns of files not to serach for. Defaults to [None]

    """

    path = dir_path
    inclusive_path_patterns = patterns
    exclusive_path_patterns = exclusive_patterns
    all_files = pathlib.Path(dir_path)

    filtered_set = set()

    for pattern in inclusive_path_patterns:
        filtered_set = filtered_set.union(set(all_files.rglob(pattern)))

    for exclusive in exclusive_path_patterns:
        if exclusive == None:

            filtered_set = (file for file in filtered_set)

        else:

            filtered_set = filtered_set - set(all_files.rglob(exclusive))

    # filtered_gen = (file for file in np.sort(np.array(list(filtered_set))))

    return sorted(list(filtered_set))


In [ ]:
from dataset import SSD_Dataset

In [ ]:
target_path_list = find_files('/home/sk/Rewire_Image/Rewire_original_models/c-fos/', ['*.json'])
img_path_list = find_files('/home/sk/Rewire_Image/Rewire_original_models/c-fos/', ['*.jpg'])

In [ ]:
ssd_dataset= SSD_Dataset(img_path_list=img_path_list, target_path_list=target_path_list)

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
test = DataLoader(ssd_dataset)

In [ ]:
aaa = SSD_dataset[0]

In [ ]:
import matplotlib.pylab as plt
plt.imshow(aaa[0])

In [ ]:
aug = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.PadIfNeeded(min_height= patch_size, min_width=patch_size, p=1.0),
    A.RandomResizedCrop(height=patch_size, width=patch_size, p = 0.3),
    A.Rotate(crop_border=False),
    A.ColorJitter(brightness= .2, contrast=0, saturation=0, hue= .5),
    A.RandomBrightnessContrast(p=0.3)
])

In [ ]:
def make_dataset(img_data_path, mask_data_path, img_aug, edge_weight):
    train_X_data = find_files(img_data_path, ['*.jpg', '*.png', '*.tif', '*.tiff', '*.npy'])
    train_y_data = find_files(mask_data_path, ['*.jpg', '*.png', '*.tif', '*.tiff', '*.npy'])

    ma = [x for x in train_y_data]
    im = [x for x in train_X_data]

    dataset = Dataset(im, ma, img_aug= img_aug, edge_weight=False)

    return dataset